In [1]:
from __future__ import division
import pickle
import re
import os, os.path
import pandas as pd
import pickle
import sys
from sklearn.cross_validation import train_test_split
from sklearn.metrics import log_loss, confusion_matrix
import io

In [2]:
#output_path='dict_full' #where to store the output
label_csv_path='../trainLabels.csv' #path to csv file mapping filenames to labels
data_path = '../train/' #path to the data

dump_path = '../new_opcodes.txt'
unpickled_file=open(dump_path,'rb')
unpickled_dict=pickle.load(unpickled_file)
raw_opcode_list=unpickled_dict.keys() #list of opcodes to filter
#raw_opcode_list=['mov','add','sub','imul'] #list of opcodes to filter

In [3]:
opcode_dict_l=list()
for i in range(0,9):
    opcode_dict=dict()
    for item in raw_opcode_list:
        formatted_opcode=' '+item+' '
        opcode_dict[formatted_opcode]=0
    opcode_dict_l.append(opcode_dict)

def count_opcodes(directory_path,filename):
    #byte_limit=100
    new_opcode_dict=dict()
    # print opcode_dict
    for key in opcode_dict.keys():
        new_opcode_dict[key] = 0
    try:
        path=directory_path+filename
        openfile = io.open(path,'r',encoding='latin-1')
        #openfile = open(path,'rb')
        l_lines=openfile.readlines()
        count=0
        lc=0
        for line in l_lines:
            #print ("line found")
            opcode_group = re.search('\s\s\s[a-z][a-z]+\s\s\s',line)
            if opcode_group:
                opcode = opcode_group.group()
                opcode = opcode.strip()
                opcode=' '+opcode+' '
                if opcode in new_opcode_dict.keys():
                    #print ("key found")
                    new_opcode_dict[opcode] += 1
                
        openfile.close()
    except IOError:
        print ("Could not open file!")
    total=1
    '''
    for opcode in new_opcode_dict.keys():
        total+=new_opcode_dict[opcode]
    # print (new_opcode_dict)
    for opcode in new_opcode_dict.keys():
        new_opcode_dict[opcode] = new_opcode_dict[opcode]/total
    #print new_opcode_dict
    '''
    return new_opcode_dict

In [4]:
from sklearn.cross_validation import cross_val_score, KFold
from sklearn.linear_model import LogisticRegression

import numpy as np
import os.path

dt = np.dtype([('Id', 'a30'), ('Class', 'u2')])
data = np.loadtxt(label_csv_path, skiprows=1, delimiter = ',', dtype=dt)

X = np.zeros((data.shape[0], len(raw_opcode_list)))
Y = data['Class']

for i, (Id, Class) in enumerate(data):
    #print "iterating"
    asmFile = Id[1:-1]+'.asm'
    countDict = count_opcodes(data_path,asmFile)
    j = 0
    for key in countDict.keys():
        #print countDict[key]
        X[i][j] = countDict[key]
        j = j + 1
    print i, "I am alive"
    if i>100:
        break

0 I am alive
1 I am alive
2 I am alive
3 I am alive
4 I am alive
5 I am alive
6 I am alive
7 I am alive
8 I am alive
9 I am alive
10 I am alive
11 I am alive
12 I am alive
13 I am alive
14 I am alive
15 I am alive
16 I am alive
17 I am alive
18 I am alive
19 I am alive
20 I am alive
21 I am alive
22 I am alive
23 I am alive
24 I am alive
25 I am alive
26 I am alive
27 I am alive
28 I am alive
29 I am alive
30 I am alive
31 I am alive
32 I am alive
33 I am alive
34 I am alive
35 I am alive
36 I am alive
37 I am alive
38 I am alive
39 I am alive
40 I am alive
41 I am alive
42 I am alive
43 I am alive
44 I am alive
45 I am alive
46 I am alive
47 I am alive
48 I am alive
49 I am alive
50 I am alive
51 I am alive
52 I am alive
53 I am alive
54 I am alive
55 I am alive
56 I am alive
57 I am alive
58 I am alive
59 I am alive
60 I am alive
61 I am alive
62 I am alive
63 I am alive
64 I am alive
65 I am alive
66 I am alive
67 I am alive
68 I am alive
69 I am alive
70 I am alive
71 I am alive
72

In [12]:
def trainCV(X,y, clf):
 
    # Construct a kfolds object
    kf = KFold(len(y),n_folds=3,shuffle=True)
    #y_prob = np.zeros((len(y),3))
    y_pred = np.zeros(len(y))

    avLoss = 0
     
    # Iterate through folds
    for train_index, test_index in kf:
        X_train, X_test = X[train_index], X[test_index]
        #print X_train
        #print "test", test_index
        y_train = y[train_index]
 
        clf.fit(X_train,y_train)
    #print clf.get_params()
    print clf.coef_, clf.intercept_
    predictions = clf.predict_proba(X_test)
    #print "predictions", predictions
    #y_prob[test_index] = predictions
    y_pred[test_index] = clf.predict(X_test)
    y_test = y[test_index]
    #print type(y_test), type(y_pred[test_index])
    logLoss = log_loss(y_test, predictions)
    print 'Log loss is %.3f' % logLoss
    avLoss = avLoss + logLoss
    avLoss = avLoss/len(kf)
    print 'Average Log Loss is %.3f' %avLoss
    return avLoss

In [13]:
cList = [1e3,1e2,10,1,0.1,0.01,0.001]
minError = 100
minC = cList[0]
for cVal in cList:
    clf = LogisticRegression(C=cVal)
    error = trainCV(X,Y,clf)
    print 'current Log loss is %.3f' % error
    print 'current C is %.3f' % cVal
    if error < minError:
        minError = error
        minC = cVal
print 'Min log loss is %.3f' % minError
print 'Min C is %.3f' % minC

[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]] [-1.78517153 -1.22677185 -0.99771856 -3.07797687 -5.55192962 -2.60209131
 -3.22894206 -2.04648103 -2.30377396]
Log loss is 1.894
Average Log Loss is 0.631
current Log loss is 0.631
current C is 1000.000
[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]] [-1.82036752 -1.21265039 -0.98094866 -3.09440296 -5.51499124 -2.57025836
 -3.26715924 -2.08096307 -2.28220462]
Log loss is 1.902
Average Log Loss is 0.634
current Log loss is 0.634
current C is 100.000
[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  

In [5]:
X[2]

array([  4.61000000e+02,   0.00000000e+00,   6.00000000e+00,
         0.00000000e+00,   8.00000000e+00,   6.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   6.00000000e+00,   0.00000000e+00,
         3.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         1.70000000e+02,   7.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         1.20000000e+01,   7.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   2.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   7.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         9.15000000e+02,

In [21]:
s = " dfd "
s= s.strip()
s

'dfd'

In [ ]:
classifiers = [
    KNeighborsClassifier(20),
    #SVC(kernel="linear", C=0.025, probability=True),
    #SVC(gamma=2, C=1, probability=True),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=10, n_estimators=100, max_features=2)]
    #AdaBoostClassifier(),
    #GaussianNB(),
    #LDA()]